In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np

In [2]:

x = torch.tensor(1, dtype=torch.float).requires_grad_(True)
y = torch.tensor(1, dtype=torch.float).requires_grad_(True)
z = torch.tensor(1, dtype=torch.float).requires_grad_(True)
def f(x,y,z):
    return z * (x + y)
o = f(x,y,z)
o.backward()
x.grad
y.grad
z.grad

tensor(2.)

In [3]:
w = torch.tensor([2, -3, -3], dtype=float, requires_grad = True)
x = torch.tensor([-1, -2], dtype=float, requires_grad = True)

def f(x,w):
  return 1/(torch.exp(-torch.dot(w[0:2],x)-w[2])+1)

o = f(x,w)
o.backward()
print(w.grad)
print(x.grad)


tensor([-0.1966, -0.3932,  0.1966], dtype=torch.float64)
tensor([ 0.3932, -0.5898], dtype=torch.float64)


In [4]:
import torch
import torchvision
from torchvision import transforms, datasets

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
train = torchvision.datasets.MNIST('', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor()
                       ]))

test = torchvision.datasets.MNIST('', train=False, download=True,transform=transforms.Compose([transforms.ToTensor()]))
trainset = torch.utils.data.DataLoader(train, batch_size=10, shuffle=True)
testset = torch.utils.data.DataLoader(test, batch_size=10, shuffle=False)

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 10)
        self.dropout =nn.Dropout(0.25)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        # x = self.dropout(x)
        x = F.relu(self.fc2(x))
        # x = self.dropout(x)
        x = F.relu(self.fc3(x))
        # x = self.dropout(x)
        x = self.fc4(x)
        return F.log_softmax(x, dim=1) 

    def compute_l1_loss(self, w):
      return torch.abs(w).sum()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = Net().to(device)

In [6]:
loss_criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=0.0001)

In [ ]:
torch.cuda.empty_cache()
for epoch in range(5): 
    for data in trainset:  
        X, y = data
        X = X.to(device)
        y = y.to(device) 
        optimizer.zero_grad()  
        output = net(X.view(-1,784))  
        loss = loss_criterion(output, y)  

        # Compute L1 loss component
        l1_weight = 0.001
        l1_parameters = []
        for parameter in net.fc1.parameters():
          l1_parameters.append(parameter.view(-1))

        l1 = l1_weight * net.compute_l1_loss(torch.cat(l1_parameters))

        # Add L1 loss component
        loss += l1

        # Backpropergation 
        loss.backward()  
        optimizer.step()  
    print(loss) 

tensor(0.7596, grad_fn=<AddBackward0>)
tensor(0.4491, grad_fn=<AddBackward0>)
tensor(0.5212, grad_fn=<AddBackward0>)
tensor(0.2125, grad_fn=<AddBackward0>)
tensor(0.5675, grad_fn=<AddBackward0>)


In [8]:
net.fc1.weight

Parameter containing:
tensor([[-1.1646e-05, -7.4986e-06, -2.4820e-05,  ...,  7.0655e-06,
          7.2518e-06,  4.3654e-05],
        [-1.5048e-05,  2.7220e-06,  2.6725e-05,  ...,  1.5126e-06,
          1.6211e-05,  8.0420e-06],
        [ 4.4129e-06,  2.7076e-06, -1.3715e-05,  ...,  1.5589e-05,
          2.6520e-06,  3.5886e-05],
        ...,
        [ 9.9537e-06,  3.0904e-06, -1.2866e-05,  ..., -8.6891e-06,
         -3.0117e-05,  1.3274e-05],
        [ 2.9324e-05,  2.4141e-05, -6.9225e-06,  ..., -5.4601e-06,
          8.8380e-06, -1.1843e-05],
        [ 1.3003e-07, -8.4465e-06,  7.6089e-06,  ..., -3.8580e-05,
         -6.3762e-06, -8.8704e-06]], requires_grad=True)

In [9]:
correct = 0
total = 0

with torch.no_grad():
    for data in testset:
        X, y = data
        X = X.to(device)
        y = y.to(device)
        output = net(X.view(-1,784))
        
        for idx, i in enumerate(output):
          if torch.argmax(i) == y[idx]:
            correct += 1
          total += 1

print("Accuracy: ", round(correct/total, 2))

Accuracy:  0.94
